wordpress 4.
pip3 install: python-wordpress-xmlrpc

In [7]:
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import posts
from wordpress_xmlrpc.methods import taxonomies
from wordpress_xmlrpc import WordPressTerm
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc.methods import media, posts
import mysql.connector as mariadb
from mysql.connector import Error
import os, glob

path = '/mnt/e/image/'
wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')

# open a connection of mariadb
        mariadb_connection = mariadb.connect(
            host = 'localhost',
            user = 'root',
            password = '',
            database = 'douban_books'
        )

# get mariadb cursor
cursor = mariadb_connection.cursor()

for jpgfile in glob.iglob(os.path.join(path, "*.jpg")):
    # prepare metadata
    data = {
            'name': jpgfile.split('/')[-1],
            'type': 'image/jpeg',  # mimetype
    }
    # read the binary file and let the XMLRPC library encode it into base64
    with open(jpgfile, 'rb') as img:
            data['bits'] = xmlrpc_client.Binary(img.read())


    try:
        # get rows of table
        sql = "select * from tag_novel where ";
        cursor.execute(sql)
        records = cursor.fetchall()
        print("Total number of rows: ", cursor.rowcount)
        for row in records:
            filename = path + row[5].split('/')[-1]
            urllib.request.urlretrieve(row[5], filename=filename)
        #end for row in records
        cursor.close()

    except Error as e:
        print("Error while connecting to mariaDB", e) 
        
    #end try

    attachment_id = response['id']
    post = WordPressPost()
    post.title = '1942—1945：我的上海沦陷生活'
    post.content = '1942—1945：我的上海沦陷生活，颜滨 著、采金 整理，人民出版社，59.00元'
    post.post_status = 'publish'  #文章状态，不写默认是草稿，private表示私密的，draft表示草稿，publish表示发布
    post.terms_names = {
        'post_tag': ['Novel', 'firstpost'], #文章所属标签，没有则自动创建
        'category': ['Books', 'Tests'] #文章所属分类，没有则自动创建
     }
    post.thumbnail = attachment_id #缩略图的id
    post.id = wp.call(posts.NewPost(post))
    
    break
#end for

# close a connection of mariadb
if(mariadb_connection.is_connected()):
    mariadb_connection.close()
    print("mariaDB connection is closed.")

In [ ]:
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo

wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')
wp.call(GetPosts())

wp.call(GetUserInfo())

post = WordPressPost()
post.title = 'My new title'
post.content = 'This is the body of my new post.'
post.terms_names = {
    'post_tag': ['test', 'firstpost'],
    'category': ['Introductions', 'Tests']
}
wp.call(NewPost(post))